We demonstrate the same frameworks but replacing the embedding scheme to amplitude encoding which is given from the
[skeleton code](./rigetti_resources/Amplitude%20Encoding.ipynb).

The detail implementation can be found in [pyquil_circuits.py](./pyquil_circuits.py#L161).


In [1]:
import numpy as np
from pyquil_circuits import AmplitudeEncoding, VariationalCircuit
from QVC import PyquilVariationalClassifier
from utils import load_data, bf, zero_padding

The number of amplitudes should be the power of 2. In case the feature dimension is not the power of 2,
we put zero padding.

In [2]:
qfm = AmplitudeEncoding(num_amps=4)
print(qfm.circuit)

DECLARE beta REAL[3]
RY(beta[2]) 1
CONTROLLED RY(beta[1]) 1 0
X 1
CONTROLLED RY(beta[0]) 1 0
X 1
DECLARE ro BIT[2]
MEASURE 0 ro[0]
MEASURE 1 ro[1]



Declare the variational circuit. We put 4 amplitudes into two qubits.

In [3]:
vc = VariationalCircuit(num_qubits=2, rep=2)
print(vc.circuit)

DECLARE param REAL[8]
RY(param[0]) 0
RY(param[1]) 1
RZ(param[2]) 0
RZ(param[3]) 1
CZ 0 1
RY(param[4]) 0
RY(param[5]) 1
RZ(param[6]) 0
RZ(param[7]) 1
CZ 0 1
DECLARE ro BIT[2]
MEASURE 0 ro[0]
MEASURE 1 ro[1]



We load the data with the number of principal component of 3.
Also, we divide the data into the training data(80%) and test data(20%). Specifying the random seed, we can repeat the experiment with the same condition.
The number of amplitudes is 4 in qfm, we put zero padding.

In [4]:
seed = 30
train_data, train_labels, test_data, test_labels = load_data(test_size=0.2, num_PCs=3, seed=seed)
print(f"# of training data = {len(train_data)}")
print(f"# of test data     = {len(test_data)}")
print(f"dimension of data  = {train_data.shape[1]}")
train_data = zero_padding(train_data, 4)
test_data = zero_padding(test_data, 4)
print(f"dimension of padded data  = {train_data.shape[1]}")

# of training data = 81
# of test data     = 21
dimension of data  = 3
dimension of padded data  = 4


In [5]:
qvc = PyquilVariationalClassifier(qfm, vc, bool_ftn=bf, use_bias=False)

Start training. To reduce the time, we limited the spsa iterations to 20.
To monitor the process, you can use `tensorboard`.
`tensorboard --logdir=./runs/zzzpfm_c12v3_zzzpfm_c12v3_pyquil_amp_enc`

In [ ]:
print("Start training")

point, value, nfev = qvc.train(train_data, (-1) ** train_labels, 'zzzpfm_c12v3_pyquil_amp_enc',
                               test_data=test_data, test_label=(-1) ** test_labels,
                               spsa_maxiter=20)
print("Training Done")
print(f"optimal params       = {point}")
print(f"final training loss  = {value}")
print(f"function evaluations = {nfev}")

Start training


Save the training result

In [ ]:
np.save('./npy_files/TrainData_zzpfmc12_pyquil_amp_enc.npy', train_data)
np.save('./npy_files/TestData_zzpfmc12_pyquil_amp_enc.npy', test_data)
np.save('./npy_files/TrainLabels_zzpfmc12_pyquil_amp_enc.npy', train_labels)
np.save('./npy_files/TestLabels_zzpfmc12_pyquil_amp_enc.npy', test_labels)
np.save('./npy_files/Optimal_param_zzpfmc12_pyquil_amp_enc.npy', point)